In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.densenet import DenseNet121
from keras import metrics

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/MyDrive/Monkeypox/dataset/train"

Mounted at /content/gdrive
ls: cannot access '/content/gdrive/MyDrive/Monkeypox/dataset/train': No such file or directory


In [3]:
# from tensorflow.keras.mixed_precision import experimental as mixed_precision
from tensorflow.python.keras.mixed_precision.policy import set_global_policy
set_global_policy('mixed_float16')
# set_global_policy(policy)
# print('Compute dtype: %s' % policy.compute_dtype)
# print('Variable dtype: %s' % policy.variable_dtype)

In [4]:
train_dir = "/content/drive/MyDrive/Monkeypox/dataset/train"
test_dir = "/content/drive/MyDrive/Monkeypox/dataset/test"

In [5]:
model1 = DenseNet121(include_top=False,input_shape=(224, 224, 3), weights='imagenet')

29084464/29084464 [==============================] - 2s 0us/step


In [6]:
input_shape = (224,224)

In [7]:
datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=False,
                                  vertical_flip=True)


datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 32
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)


FileNotFoundError: ignored

In [ ]:
def print_layer_trainable():
    for layer in model1.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


print_layer_trainable()

In [ ]:
transfer_layer = model1.get_layer('conv5_block16_concat')
conv_model = Model(inputs=model1.input, outputs=transfer_layer.output)

In [ ]:
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


print_layer_trainable()

In [ ]:
#for layer in conv_model.layers:
#    layer.trainable = False

# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.

new_model.add(Dense(1024, activation='relu'))
# new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))
# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
#new_model.add(Dense(512, activation='relu'))


# Add the final layer for the actual classification.
new_model.add(Dense(3, activation='softmax'))

In [ ]:
optimizer = Adam()
loss = 'categorical_crossentropy'
metrics = ["categorical_accuracy"]


new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
epochs = 80

steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size
print(steps_per_epoch)
print(steps_test)

#steps_per_epoch = 20
#steps_test = 100
history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

In [ ]:
new_model.save('/content/drive/MyDrive/Monkeypox/DenseNet121.h5')

In [ ]:
y_pred = new_model.predict_generator(generator_test,steps = steps_test)

In [ ]:
predicted_class_indices=np.argmax(y_pred,axis=1)
print(predicted_class_indices)

In [ ]:
true_labels = generator_test.classes

In [ ]:
y_pred.shape

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels, predicted_class_indices))

In [ ]:
generator_test.class_indices

In [ ]:
from sklearn.metrics import confusion_matrix
array = confusion_matrix(true_labels, predicted_class_indices)

In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
df_cm = pd.DataFrame(array, index = ["chickenpox+measles", "monkeypox", "normal"],
                  columns = ["chickenpox+measles", "monkeypox", "normal"])
plt.figure(figsize = (10,9))
sn.heatmap(df_cm, annot=True, cmap='Blues', fmt='g')

In [ ]:
import matplotlib.pyplot as plt
print(history.history)
print(history.history['categorical_accuracy'])
print(history.history['val_categorical_accuracy'])
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
# plt.plot([0.9730,0.9416,0.9319,0.9611,0.9632,0.9739,0.9769,0.9781,0.9798,0.9816])
# plt.plot([0.9250,0.9610,0.9730,0.9760,0.9690,0.9670,0.9620,0.9680,0.9650,0.9620])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
print(history.history['loss'])
print(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
# plt.plot([0.3015,0.1494,0.1164,0.0994,0.0858,0.0766,0.0689,0.0649,0.0588,0.0526])
# plt.plot([0.2164,0.1063,0.0831,0.0733,0.0940,0.1094,0.1479,0.1522,0.1211,0.1249])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.show()